# Hello World with **the Memex API**


### Install Memex package


In [ ]:
!pip install -q memexdata

#### Set your instance and API information

First, set the variables for your Memex account:

- `MEMEX_INSTANCE_URL`: The URL to your instance.
- `MEMEX_API_KEY`: You can find it under your profile bubble at the top right corner of the Memex UI after you log-in.


In [ ]:
MEMEX_INSTANCE_URL = "https://[YOUR_INSTANCE].memexdata.com"
MEMEX_API_KEY = [YOUR_API_KEY]

#### Connect to Memex


In [ ]:
from memex import MemexSession

mx = MemexSession(MEMEX_INSTANCE_URL, api_key=MEMEX_API_KEY)

Anything we create or save via the API will also be reflected in the [Memex UI](https://[YOUR_INSTANCE].memexdata.com)


### Datasets

List available datasets


In [ ]:
mx.list_datasets()

Run a query against a dataset


In [ ]:
mx.query("SELECT * FROM ancient_civilizations LIMIT 10")

## UDFs

Memex allow you to combine SQL queries with UDFs (User Defined Functions). UDFs can be written in Python or in natural language!

### Python UDF

Here is a simple example a Python function to check if two words are anagrams.


In [ ]:
def is_anagram(word1, word2):
    return sorted(word1) == sorted(word2)

Let’s test it on an example


In [ ]:
is_anagram("elvis", "lives")

We get back `True`. Let’s run another test:


In [ ]:
is_anagram("elvis", "dead")

We get `False`.

To make this function available to Memex, we just need to annotate it with the `@mx.udf` decorator


In [ ]:
@mx.udf
def is_anagram(word1: str, word2: str) -> bool:
    return sorted(word1) == sorted(word2)

Now we can use this function in a SQL query in Memex.

We can call it with the following syntax with a SQL query:


In [ ]:
mx.query("SELECT is_anagram('elvis', 'lives')")

{% hint style="info" %}
💡 **Note:** Once the function is defined, it is automatically available in Memex and you can use it in any SQL query in the [Memex UI](https://[YOUR_INSTANCE].memexdata.com) as well.
{% endhint %}


### Natural Language UDF

Memex also allows you to define UDFs using just natural language. To do that, you define a Python function with a docstring that contains the prompt for the LLM (Large Language Model) and annotate it with the `@mx.prompt` decorator.


In [ ]:
@mx.prompt
def generate_anagram(n: int) -> str:
    """Generate {n} pair of anagrams."""

Which can then be called within a SQL query:


In [ ]:
mx.query("SELECT generate_anagram(5)")

This query would return something like this:

<table data-full-width="false"><thead><tr><th width="196"></th><th>generate_anagram(5)</th></tr></thead><tbody><tr><td>0</td><td>1. laps - pals\n2. store - roset\n3. listen - ...</td></tr></tbody></table>

Behind the scene this translates to a call to a LLM with a given prompt. The prompt (or prompt template), defined in the Python docstring of the function, and the function parameters are evaluated with the prompt to generate the final prompt for the LLM.

We can do a bit of post-processing to get a list of rows in table structure


mx.query("SELECT unnest(split(generate_anagram(5), '\\n')) as anagram")


Which returns something like this

<table><thead><tr><th width="76"></th><th width="194">anagram</th></tr></thead><tbody><tr><td>0</td><td>1. laps -pals</td></tr><tr><td>1</td><td>2. store - roset</td></tr><tr><td>2</td><td>3. listen - silent</td></tr><tr><td>3</td><td>4. evil - live</td></tr><tr><td>4</td><td>5. heart - earth</td></tr></tbody></table>

Within the function call we can specify the use a different LLM model, change the temperature, max_tokens, etc. By default, `gpt-3.5-turbo` is used.


mx.query("SELECT unnest(split(generate_anagram(5), '\\n')) as anagram",
model='gpt-4', temperature=0.2, max_tokens=256)


Now that we’ve gone through a simple example, let's try a more complex one


In [ ]:
@mx.prompt
def ancient_civilization(n: int) -> str:
    """Give me {n} names of random ancient civilizations.
    Provide just the canonical name, and exclude qualifiers like
    "civilization", "empire", etc. FORMAT the data as a COMMA DELIMITED LIST,
    like: civ1, civ2, civ3, ..."""

mx.query("SELECT unnest(split(ancient_civilization(5), ',')) as civilization")


Which returns a table that should look like this:

<table><thead><tr><th width="100"></th><th width="196">civilization</th></tr></thead><tbody><tr><td>0</td><td>Egypt</td></tr><tr><td>1</td><td>Mesopotamia</td></tr><tr><td>2</td><td>Maya</td></tr><tr><td>3</td><td>Indus Valley</td></tr><tr><td>4</td><td>Zhou Dynasty</td></tr></tbody></table>

Now we can save the results of this query as a new table in Memex using the `save_as_table` command


In [ ]:
mx.save_as_table("ancient_civilizations", 
                 "SELECT unnest(split(ancient_civilization(20), ',')) as civilization", overwrite**=True**)

You can now go to the [Memex UI](https://[YOUR_INSTANCE].memexdata.com) and see this new table

You’ve said Hello World with the Memex API!
